# Naive Bayes

In [3]:
import pandas as pd
df=pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [5]:
target=df.Survived
inputs=df.drop(['Survived'],axis='columns')

In [6]:
dummies=pd.get_dummies(inputs.Sex)
inputs=pd.concat([inputs,dummies],axis='columns')
inputs.head()

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False
3,1,female,35.0,53.1000,True,False
4,3,male,35.0,8.0500,False,True


In [7]:
inputs=df.drop('Sex',axis='columns')

In [8]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [9]:
(inputs.columns[inputs.isna().any()]).shape[0]

1

In [10]:
mn=inputs.Age.mean()
inputs.Age=inputs.Age.fillna(mn)

In [11]:
from sklearn.model_selection import train_test_split
xtrain,xtes,ytrain,ytes=train_test_split(inputs,target,test_size=0.2)

In [12]:
from sklearn.naive_bayes import GaussianNB

In [13]:
model=GaussianNB()

In [14]:
model.fit(xtrain,ytrain)

,priors,None
,var_smoothing,1e-09


In [15]:
model.score(xtes,ytes)

1.0

In [16]:
xtes

,Survived,Pclass,Age,Fare
402,0,3,21.000000,9.8250
228,0,2,18.000000,13.0000
800,0,2,34.000000,13.0000
1,1,1,38.000000,71.2833
163,0,3,17.000000,8.6625
...,...,...,...,...
407,1,2,3.000000,18.7500
739,0,3,29.699118,7.8958
426,1,2,28.000000,26.0000
59,0,3,11.000000,46.9000


In [17]:
ytes

402    0
228    0
800    0
1      1
163    0
      ..
407    1
739    0
426    1
59     0
507    1
Name: Survived, Length: 179, dtype: int64

In [18]:
model.predict(xtes[:10])

array([0, 0, 0, 1, 0, 0, 1, 0, 0, 1], dtype=int64)

In [19]:
ytes[:10]

402    0
228    0
800    0
1      1
163    0
188    0
641    1
365    0
841    0
247    1
Name: Survived, dtype: int64

In [20]:
df2=pd.read_csv('spam.csv')

In [21]:
df2.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [22]:
df2['spam']=df2.Category.apply(lambda x: 1 if x=='spam' else 0)
df2.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [23]:
xtrain,xtes,ytrain,ytes=train_test_split(df2.Message,df2.spam,test_size=0.25)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
v= CountVectorizer()
xtrain_count=v.fit_transform(xtrain.values)
xtrain_count.toarray()[:3]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(xtrain_count,ytrain)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [26]:
emails=[
    'hey man lets play football',
    'upto 20% discount on sportware, exclusive offer just fro you'
]
emails_count=v.transform(emails)
model.predict(emails_count)

array([0, 1], dtype=int64)

In [30]:
xtes_count=v.transform(xtes)
model.score(xtes_count,ytes)

0.9856424982053122

In [31]:
## sklearn pipeline
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ('vectorizer',CountVectorizer()),
    ('nb',MultinomialNB())
])

In [32]:
clf.fit(xtrain,ytrain)

,steps,"[('vectorizer', ...), ('nb', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [33]:
clf.predict(xtes)[:5]

array([0, 1, 0, 0, 0], dtype=int64)